## Manufactured Solutions for the Stokes-Biot system

A manufactured solution for the couple problem has to fullfill the interface conditions:

\begin{align}
        \mathbf{u} \cdot \mathbf{n} &= \left( \partial_t \mathbf{d} - \frac{\kappa}{\mu_f} \nabla p_p \right) \cdot \mathbf{n} &\text{(continuity of normal flux)} \\
        \left( 2 \mu_f \epsilon (\mathbf{u}) - p_F \mathbf{I}  \right) \mathbf{n}  &= \left( 2 \mu_s \epsilon(\mathbf{d}) - \phi \mathbf{I}  \right) \mathbf{n}  &\text{(momentum conservation)} \\
       -\mathbf{n} \cdot \left( 2 \mu_f \epsilon (\mathbf{u}) - p_F \mathbf{I}  \right) \mathbf{n}  &= p_p   &\text{(balance of fluid normal stress)} \\
       -\mathbf{n} \cdot \left( 2 \mu_f \epsilon (\mathbf{u}) - p_F \mathbf{I}  \right) \mathbf{t}  &= \frac{\gamma \mu_f}{\sqrt{\kappa}} \left( \mathbf{u} - \partial_t \mathbf{d} \right) \cdot \mathbf{t}   &\text{(Beavers-Joseph-Saffman condition)} 
    \end{align}   
    
Since a solution, that fullfills these conditions is hard to manufacture, we split the problem in a stationary problem for testing the spatial discretization and a spatially uniform problem to test the temporal discretization.

For further simplification, we assume a 2D geometry with straight vertical interface:
$$ \mathbf{n} = (1,0)^T$$
and
$$\mathbf{t} = (0,1)^T$$

## spatially uniform case

\begin{align}
u \cdot n = \partial_t d \cdot n \\
p_F = \phi = p_P \\
0 = \frac{\gamma \mu_f}{\sqrt{\kappa}} u- \partial_t d ) \cdot n \\
\Rightarrow u  = \partial_t d
\end{align}

### Stationary Case
If we remove all time derivatives in the equations above, we end up with the following system:
\begin{align}
        \mathbf{u} \cdot \mathbf{n} &= \left( - \frac{\kappa}{\mu_f} \nabla p_p \right) \cdot \mathbf{n} &\text{(continuity of normal flux)} \\
        \left( 2 \mu_f \epsilon (\mathbf{u}) - p_F \mathbf{I}  \right) \mathbf{n}  &= \left( 2 \mu_s \epsilon(\mathbf{d}) - \phi \mathbf{I}  \right) \mathbf{n}  &\text{(momentum conservation)} \\
       -\mathbf{n} \cdot \left( 2 \mu_f \epsilon (\mathbf{u}) - p_F \mathbf{I}  \right) \mathbf{n}  &= p_p   &\text{(balance of fluid normal stress)} \\
       -\mathbf{n} \cdot \left( 2 \mu_f \epsilon (\mathbf{u}) - p_F \mathbf{I}  \right) \mathbf{t}  &= \frac{\gamma \mu_f}{\sqrt{\kappa}} \left( \mathbf{u} \right) \cdot \mathbf{t}   &\text{(Beavers-Joseph-Saffman condition)} 
    \end{align}   

In [ ]:
%load_ext autoreload
%autoreload 2
import sympy as sym
from sympy import pi
import numpy as np
from sympy import init_printing
import matplotlib.pyplot as plt
import yaml

import json2latex
from contextlib import redirect_stdout
from braininversion.BiotStokesConvergence import (compute_spatial_mm,generate_expression,
                                                  compute_forcing_terms,check_interface_conditions,
                                                  boundary_id, compute_spatial_convergence, names,
                                                  compute_order, generate_mesh)
plt.style.use('bmh')

T = 0.8
num_steps = 4
dt = T/num_steps
times = np.linspace(0, T, num_steps + 1)
figsize = (6,6)
resolutions = [4,8,16,32,64, 128]

material_parameter = dict()
##E = 1
nu = 0.4999

u_degree = 2
p_degree = 1

material_parameter["c"] = 1.1
material_parameter["kappa"] = 1.3
material_parameter["lambda"] =  11 # nu*E/((1.0-2.0*nu)*(1.0+nu))
material_parameter["mu_s"] =   5.3# E/(2.0*(1.0+nu))
material_parameter["rho_s"] = 1.2
material_parameter["mu_f"] = 0.9
material_parameter["rho_f"] = 1.5
material_parameter["alpha"] = 1.0
material_parameter["gamma"] = 0.9
material_parameter["lmbda"] = material_parameter["lambda"]

x,y,z = sym.symbols("x, y, z")
t = sym.symbols("t")

interf_x = 0.0

a = 2
u_1 = +sym.sin(a*pi*x)*sym.cos(a*pi*y)# + sym.cos(2*pi*y)
u_2 = -sym.cos(a*pi*x)*sym.sin(a*pi*y)# + sym.cos(4*pi*x)


u = sym.Matrix([u_1, u_2])#*t

d, p_p, p_f, phi = compute_spatial_mm(u, interf_x)

check_interface_conditions(u, p_f, d, p_p, interf_x)
f_fluid, f_porous, g_source = compute_forcing_terms(u, p_f, d, p_p)
expressions = {"pF":p_f, "pP":p_p, "d":d, "u":u, "phi":phi,
              "f_fluid":f_fluid, "f_porous":f_porous, "g_source":g_source}

for name, exp in expressions.items():
    with open(f'../results/MMS/spatial_expressions/{name}.tex', 'w') as f:
        with redirect_stdout(f):
            print(sym.latex(exp, long_frac_ratio=5.0))

In [ ]:
L2_error, H1_error, num_results, h, exact_sols = compute_spatial_convergence(resolutions, T, num_steps, 
                                                   u, p_f, d, p_p, phi, material_parameter,
                                                   f_fluid, f_porous, g_source,
                                                   u_degree, p_degree)

In [ ]:
L2_rates = {}
H1_rates = {}
for n in names:
    L2_rates[n] = compute_order(L2_error[n], h).tolist()
    H1_rates[n] = compute_order(H1_error[n], h).tolist()

In [ ]:
h = np.array(h)
plt.figure(figsize=figsize)
for n in names:
    plt.loglog(1/h, L2_error[n], "*-", label=n)
plt.loglog(1/np.array(h), 0.5e1*h**2 , "-", label="$O(h^2)$")
plt.loglog(1/np.array(h), 1e0*h**3 , "-", label="$O(h^3)$")

ticks = [5,10,20,50]
plt.legend()
plt.grid(which="both")
plt.xlabel("1/h")
plt.ylabel("L2 error")
plt.xticks( ticks, ticks)
plt.tight_layout()
plt.savefig("../results/MMS/spatial_L2.pdf")
#plt.savefig("../results/MMS/spatial_L2.pgf")

In [ ]:
plt.figure(figsize=figsize)
for n in names:
    plt.loglog(1/h, H1_error[n], "*-", label=n)
plt.loglog(1/np.array(h), 1e1*h**1 , "-", label="$O(h^1)$")
plt.loglog(1/np.array(h), 1e0*h**2 , "-", label="$O(h^2)$")

ticks = [5,10,20,50]
plt.legend()
plt.grid(which="both")
plt.xlabel("1/h")
plt.ylabel("H1 error")
plt.xticks( ticks, ticks)
plt.tight_layout()
plt.savefig("../results/MMS/spatial_H1.pdf")
#plt.savefig("../results/MMS/spatial_H1.pgf")

In [ ]:
conv_results = {"L2rates":L2_rates,
                "H1rates":H1_rates,
                "T":T,
                "dt":dt,
                "L2error":L2_error,
                "H1error":H1_error,
                "resolutions":resolutions,
                "material_parameter":material_parameter}


with open('../results/MMS/spatial_conv.yml', 'w') as f:
    yaml.dump(conv_results, f)

In [ ]:
from fenics import *

mesh, boundaries, subdomains = generate_mesh(resolutions[-1])

W = FunctionSpace(mesh, "CG", p_degree)
V = VectorFunctionSpace(mesh, "CG", u_degree)


#plt.figure(figsize=fig_size)
#cb = plot(sqrt(inner(diff, diff)), mesh=mesh)
#cb = plot(diff, mesh=mesh)
#plot(mesh)
#plt.colorbar(cb)
#plt.xlim(xlims)
#assemble(sqrt(inner(diff,diff))*ds(domain=mesh))

In [ ]:

L2_error, H1_error, num_results, h, exact_sols = compute_spatial_convergence([12], T, num_steps, 
                                                   u, p_f, d, p_p, phi, material_parameter,
                                                   f_fluid, f_porous, g_source,
                                                   u_degree, p_degree)

In [ ]:
W = FunctionSpace(mesh, "DG", p_degree)
i = -1

def plot_comparison(i, name_f, name_p, W, abs_diff=True, figsize=(5,3)):
    xlims = (-1,1)

    exact_sol_f = exact_sols[name_f]
    exact_sol_f.t = times[i]
    exact_sol_p = exact_sols[name_p]
    exact_sol_p.t = times[i]
    fluid_filter = Expression("(x[0] <= 0) ? 1 : 0",degree=2)
    exact_sol = project(exact_sol_f*fluid_filter + (1-fluid_filter)*exact_sol_p , W)

    num_sol = project(num_results[name_f][i]*fluid_filter + (1-fluid_filter)*num_results[name_p][i] , W)
    diff = num_sol - exact_sol

    plt.figure(figsize=figsize)
    cb = plot(exact_sol, mesh=mesh)
    plt.colorbar(cb, shrink=0.75)
    plt.xlim(xlims)
    #plt.title("exact")
    plt.title("$\Omega_F $                            $ \Omega_P$")
    plt.tight_layout()
    plt.savefig(f"../results/MMS/{name_f}_{name_p}_exact.pdf")

    plt.figure(figsize=figsize)
    cb = plot(num_sol, mesh=mesh)
    plt.colorbar(cb, shrink=0.75)
    plt.xlim(xlims)
    #plt.title("numerical")
    plt.title("$\Omega_F $                            $ \Omega_P$")
    plt.tight_layout()
    plt.savefig(f"../results/MMS/{name_f}_{name_p}_num.pdf")

    plt.figure(figsize=figsize)
    if abs_diff:
        cb = plot(abs(diff), mesh=mesh)
    else:
        cb = plot(diff, mesh=mesh)
    plt.colorbar(cb, shrink=0.75)
    plt.xlim(xlims)
    #plt.title("diff")
    plt.title("$\Omega_F $                            $ \Omega_P$")
    plt.tight_layout()
    plt.savefig(f"../results/MMS/{name_f}_{name_p}_diff.pdf")


In [ ]:
plot_comparison(-1, "pF", "phi", W)

In [ ]:
plot_comparison(-1, "u", "d", V, abs_diff=False)

In [ ]:
times